### Spielsteine von Punkt zu Punkt schieben

In [20]:
N_POS = 6
positions = [0, 2]  # position[i] ist Position von Stein i


def is_legal(stone, i, j):
    return (0 <= i < N_POS
            and 0 <= j < N_POS
            and i != j
            and positions[stone] == i
            and j not in positions)



def move(stone, i, j):
    if is_legal(stone, i, j):
        positions[stone] = j
        return True

In [21]:
move(1, 2, 4)  # bewege Stein 1 auf Position 4

True

In [22]:
positions  # Stein 0 ist immer nach auf 0, Stein 1 ist auf 4

[0, 4]

In [23]:
from math import sin, cos, pi


def get_points_on_circle(n, center=(0, 0), r=1):
    x, y = center
    pts = [(x + r*cos(2*i*pi/n), y - r*sin(2*i*pi/n)) for i in range(n)]
    return pts


pts = get_points_on_circle(6, (100, 100), r=100)
[(round(x), round(y)) for x, y in pts]

[(200, 100), (150, 13), (50, 13), (0, 100), (50, 187), (150, 187)]

In [58]:
from ipycanvas import MultiCanvas


stone_color = {0: 'red',
               1: 'blue',
               }

positions = [0, 1]
state = {'picked': None}
POINTS = get_points_on_circle(n=6, center=(100, 100), r=50)
RADIUS = 10


def place_stone(canvas, stone, i):
    canvas.fill_style = stone_color[stone]
    canvas.fill_circle(*POINTS[i], RADIUS)


def delete_stone(canvas, i):
    r = RADIUS + 1
    x0, y0 = POINTS[i]
    canvas.clear_rect(x0-r, y0-r, 2*r)


layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas
fg.text_align = 'center'
fg.text_baseline = 'middle'

for i, (x, y) in enumerate(POINTS):
    fg.fill_text(str(i), x, y)

for stone in stone_color:
    place_stone(bg, stone, positions[stone])

mcanvas

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [66]:
def try_move(stone, i, j):
    if positions[stone] != i:
        return
    delete_stone(bg, i)
    if move(stone, i, j):
        place_stone(bg, stone, j)
    else:
        place_stone(bg, stone, i)

In [64]:
try_move(1, 1, 4)

In [65]:
try_move(1, 4, 0)

In [3]:
from ipywidgets import Output
from ipycanvas import MultiCanvas
from IPython.display import display
import helpers as H


layout = {'border': '1px solid black'}
out = Output(layout=layout)
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas
fg.text_align = 'center'
fg.text_baseline = 'middle'

for i, (x, y) in enumerate(POINTS):
    fg.fill_text(str(i), x, y)

for stone in stone_color:
    place_stone(bg, stone)


@out.capture(clear_output=True)
def on_mouse_down(x, y):
    d, p = H.get_closest((x, y), POINTS)
    if d > RADIUS or p not in positions:
        return

    stone = positions.index(p)
    state['picked'] = stone
    delete_stone(bg, stone)
    print(f'picked stone {stone} at {p}')


def restore():
    stone = state['picked']
    place_stone(bg, stone)
    state['picked'] = None
    print(f'keep stone {stone} at {positions[stone]}')


@out.capture()
def on_mouse_up(x, y):
    if state['picked'] is None:
        return

    d, p = H.get_closest((x, y), POINTS)
    print(d, p)

    if d < RADIUS and p not in positions:
        stone = state['picked']
        positions[stone] = p
        place_stone(bg, stone)
        state['picked'] = None
        print(f'move stone {stone} to {p}')
    else:
        restore()


@out.capture()
def on_mouse_out(x, y):
    if state['picked'] is None:
        return

    restore()



mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)
mcanvas.on_mouse_out(on_mouse_out)

display(mcanvas, out)

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…